In [ ]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

# model_client = OpenAIChatCompletionClient(
#     model="gpt-4o",
#     temperature=1,
#     # api_key="sk-...", # Optional if you have an OPENAI_API_KEY env variable set.
# )

from autogen_core.models import ModelFamily
model_client = OpenAIChatCompletionClient(
    model="qwen2.5:7b",
    # model="llama3.2:3b",
    base_url="http://localhost:11434/v1",
    api_key="placeholder",
    model_info={
        "vision": False,
        "function_calling": True,
        "json_output": False,
        "family": ModelFamily.ANY,
    },
    temperature=1,
)

# Create the primary agent.
primary_agent = AssistantAgent(
    "primary",
    model_client=model_client,
    system_message="You are a helpful AI assistant.",
)

# Create the critic agent.
critic_agent = AssistantAgent(
    "critic",
    model_client=model_client,
    system_message="Provide constructive feedback for every message. Respond with 'APPROVE' to when your feedbacks are addressed.",
)

max_msg_termination = MaxMessageTermination(max_messages=3)
round_robin_team = RoundRobinGroupChat([primary_agent, critic_agent], termination_condition=max_msg_termination)

# Use asyncio.run(...) if you are running this script as a standalone script.
await Console(round_robin_team.run_stream(task="Write a unique, Haiku about the weather in Paris"), output_stats=True)

---------- user ----------
Write a unique, Haiku about the weather in Paris
---------- primary ----------
Rain kisses cobblestones,
Eiffel shimmers through soft drizzle,
Paris whispers wet.
[Prompt tokens: 31, Completion tokens: 21]
---------- critic ----------
Your Haiku captures the essence of Paris well. Consider adding a touch that contrasts the cool dampness with the city's warmth and liveliness to make it even more vivid. APPROVE
[Prompt tokens: 67, Completion tokens: 38]
---------- Summary ----------
Number of messages: 3
Finish reason: Maximum number of messages 3 reached, current message count: 3
Total prompt tokens: 98
Total completion tokens: 59
Duration: 3.19 seconds


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Write a unique, Haiku about the weather in Paris', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=31, completion_tokens=21), metadata={}, content='Rain kisses cobblestones,\nEiffel shimmers through soft drizzle,\nParis whispers wet.', type='TextMessage'), TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=67, completion_tokens=38), metadata={}, content="Your Haiku captures the essence of Paris well. Consider adding a touch that contrasts the cool dampness with the city's warmth and liveliness to make it even more vivid. APPROVE", type='TextMessage')], stop_reason='Maximum number of messages 3 reached, current message count: 3')

call approve function


In [2]:
# Use asyncio.run(...) if you are running this script as a standalone script.
await Console(round_robin_team.run_stream(), output_stats=True)

---------- primary ----------
Rain kisses cobblestones,
Eiffel shimmers through soft drizzle,
Paris sighs, alive.
[Prompt tokens: 98, Completion tokens: 23]
---------- critic ----------
Excellent! Your revisions enhance the Haiku by adding a sense of life and activity to the scene. Approve. APPROVE
[Prompt tokens: 136, Completion tokens: 26]
---------- primary ----------
Approved!

Rain kisses cobblestones,
Eiffel shimmers through soft drizzle,
Paris sighs, alive.
[Prompt tokens: 155, Completion tokens: 25]
---------- Summary ----------
Number of messages: 3
Finish reason: Maximum number of messages 3 reached, current message count: 3
Total prompt tokens: 389
Total completion tokens: 74
Duration: 5.45 seconds


TaskResult(messages=[TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=98, completion_tokens=23), metadata={}, content='Rain kisses cobblestones,\nEiffel shimmers through soft drizzle,\nParis sighs, alive.', type='TextMessage'), TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=136, completion_tokens=26), metadata={}, content='Excellent! Your revisions enhance the Haiku by adding a sense of life and activity to the scene. Approve. APPROVE', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=155, completion_tokens=25), metadata={}, content='Approved!\n\nRain kisses cobblestones,\nEiffel shimmers through soft drizzle,\nParis sighs, alive.', type='TextMessage')], stop_reason='Maximum number of messages 3 reached, current message count: 3')

In [3]:
max_msg_termination = MaxMessageTermination(max_messages=10)
text_termination = TextMentionTermination("APPROVE")
combined_termination = max_msg_termination | text_termination

round_robin_team = RoundRobinGroupChat([primary_agent, critic_agent], termination_condition=combined_termination)

# Use asyncio.run(...) if you are running this script as a standalone script.
await Console(round_robin_team.run_stream(task="Write a unique, Haiku about the weather in Paris"), output_stats=True)

---------- user ----------
Write a unique, Haiku about the weather in Paris
---------- primary ----------
Mist wraps Eiffel Tower,
Whispers through sleepy streets below,
Paris breathes, alive.
[Prompt tokens: 200, Completion tokens: 22]
---------- critic ----------
Your Haiku is beautifully crafted. To further enhance it, consider integrating the sensation of touch or smell to evoke more sensory details. APPROVE

How about:
Mist wraps Eiffel Tower,
Whispers through sleepy streets below,
Paris breathes, cold, sweet.
[Prompt tokens: 204, Completion tokens: 56]
---------- Summary ----------
Number of messages: 3
Finish reason: Text 'APPROVE' mentioned
Total prompt tokens: 404
Total completion tokens: 78
Duration: 4.95 seconds


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Write a unique, Haiku about the weather in Paris', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=200, completion_tokens=22), metadata={}, content='Mist wraps Eiffel Tower,\nWhispers through sleepy streets below,\nParis breathes, alive.', type='TextMessage'), TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=204, completion_tokens=56), metadata={}, content='Your Haiku is beautifully crafted. To further enhance it, consider integrating the sensation of touch or smell to evoke more sensory details. APPROVE\n\nHow about:\nMist wraps Eiffel Tower,\nWhispers through sleepy streets below,\nParis breathes, cold, sweet.', type='TextMessage')], stop_reason="Text 'APPROVE' mentioned")

In [6]:
from typing import Sequence

from autogen_agentchat.base import TerminatedException, TerminationCondition
from autogen_agentchat.messages import (
    AgentEvent,
    ChatMessage,
    StopMessage,
    ToolCallExecutionEvent,
)
from autogen_core import Component
from pydantic import BaseModel
from typing_extensions import Self


class FunctionCallTerminationConfig(BaseModel):
    """Configuration for the termination condition to allow for serialization
    and deserialization of the component.
    """

    function_name: str


class FunctionCallTermination(
    TerminationCondition, Component[FunctionCallTerminationConfig]
):
    """Terminate the conversation if a FunctionExecutionResult with a specific name is received."""

    component_config_schema = FunctionCallTerminationConfig
    """The schema for the component configuration."""

    def __init__(self, function_name: str) -> None:
        self._terminated = False
        self._function_name = function_name

    @property
    def terminated(self) -> bool:
        return self._terminated

    async def __call__(
        self, messages: Sequence[AgentEvent | ChatMessage]
    ) -> StopMessage | None:
        if self._terminated:
            raise TerminatedException("Termination condition has already been reached")
        for message in messages:
            if isinstance(message, ToolCallExecutionEvent):
                for execution in message.content:
                    if execution.name == self._function_name:
                        self._terminated = True
                        return StopMessage(
                            content=f"Function '{self._function_name}' was executed.",
                            source="FunctionCallTermination",
                        )
        return None

    async def reset(self) -> None:
        self._terminated = False

    def _to_config(self) -> FunctionCallTerminationConfig:
        return FunctionCallTerminationConfig(
            function_name=self._function_name,
        )

    @classmethod
    def _from_config(cls, config: FunctionCallTerminationConfig) -> Self:
        return cls(
            function_name=config.function_name,
        )


def approve() -> str:
    """Approve the message when all feedbacks have been addressed."""
    return "call approve function"


from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

# model_client = OpenAIChatCompletionClient(
#     model="gpt-4o",
#     temperature=1,
#     # api_key="sk-...", # Optional if you have an OPENAI_API_KEY env variable set.
# )

from autogen_core.models import ModelFamily
model_client = OpenAIChatCompletionClient(
    model="qwen2.5:7b",
    # model="llama3.2:3b",
    base_url="http://localhost:11434/v1",
    api_key="placeholder",
    model_info={
        "vision": False,
        "function_calling": True,
        "json_output": False,
        "family": ModelFamily.ANY,
    },
    temperature=1,
)

# Create the primary agent.
primary_agent = AssistantAgent(
    "primary",
    model_client=model_client,
    system_message="You are a helpful AI assistant.",
)

# Create the critic agent with the approve function as a tool.
critic_agent = AssistantAgent(
    "critic",
    model_client=model_client,
    tools=[approve],  # Register the approve function as a tool.
    system_message="Provide constructive feedback. Use the approve tool to approve when all feedbacks are addressed.",
)

function_call_termination = FunctionCallTermination(function_name="approve")
round_robin_team = RoundRobinGroupChat([primary_agent, critic_agent], termination_condition=function_call_termination)

# Use asyncio.run(...) if you are running this script as a standalone script.
await Console(round_robin_team.run_stream(task="Write a unique, Haiku about the weather in Paris"), output_stats=True)

---------- user ----------
Write a unique, Haiku about the weather in Paris


---------- primary ----------
Rain whispers through spires,
Seine murmurs with cobblestone mist,
Paris breathes rain.
[Prompt tokens: 31, Completion tokens: 22]
---------- critic ----------
That's beautiful! Here is another attempt:

Gentle droplets dance,
Over Louvre, they softly fall—
Paris in soft rain.
[Prompt tokens: 182, Completion tokens: 31]
---------- primary ----------
That one is lovely too!

Here’s a third attempt if you'd like:

Misty fog flows,
Eiffel shrouded in ethereal veils—
Paris in mist.
[Prompt tokens: 92, Completion tokens: 39]
---------- critic ----------
Both are excellent! Here's your final crafted Haiku for the weather in Paris:

Misty fog flows,
Eiffel shrouded in veils—soft.
Paris in mist.

Please let me know if you need any adjustments or approval to proceed!
[Prompt tokens: 260, Completion tokens: 53]
---------- primary ----------
Your Haiku is beautifully crafted and perfectly captures the essence of Parisian mist. Here's a slightly refined version for yo

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Write a unique, Haiku about the weather in Paris', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=31, completion_tokens=22), metadata={}, content='Rain whispers through spires,\nSeine murmurs with cobblestone mist,\nParis breathes rain.', type='TextMessage'), TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=182, completion_tokens=31), metadata={}, content="That's beautiful! Here is another attempt:\n\nGentle droplets dance,\nOver Louvre, they softly fall—\nParis in soft rain.", type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=92, completion_tokens=39), metadata={}, content="That one is lovely too!\n\nHere’s a third attempt if you'd like:\n\nMisty fog flows,\nEiffel shrouded in ethereal veils—\nParis in mist.", type='TextMessage'), TextMessage(source='critic', models_usage=RequestUsage(prompt_to